## **Mount Drive**

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## **Installations**

In [2]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 4.4 MB/s eta 0:00:00


## **Imports**

In [3]:
import numpy as np
import pandas as pd

In [4]:
from itertools import product, chain

In [5]:
from tqdm import tqdm

In [6]:
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots

## **Plotting functions**

In [ ]:
def training_validation_loss_plot(train_history, validation_history, metric="loss"):
    fig = go.Figure()

    fig.add_traces([
        go.Scatter(
            x=train_history.index + 1,
            y=train_history[metric],
            name=f"Training {metric.capitalize()}"),
        go.Scatter(
            x=validation_history.index + 1,
            y=validation_history[metric],
            name=f"Validation {metric.capitalize()}"),
    ])

    fig.update_layout(
        height=500, width=500,
        xaxis_title="Epoch", yaxis_title=f"{metric.capitalize()}",
        xaxis_range=[0, 19], yaxis_range=[0, 1],
        legend=dict(xanchor="right", yanchor="top")
    )

    return fig

In [8]:
def training_validation_metrics_plot(train_history, validation_history):

    fig = make_subplots(rows=2, cols=2)

    metrics = ["loss", "AUC", "f1-score", "fβ-score"]
    rows_indexes = [(i+1, i+1) for i in range(2) for j in range(2)]
    cols_indexes = [(j+1, j+1) for i in range(2) for j in range(2)]

    for i, (metric, row_indexes, col_indexes) in enumerate(zip(metrics, rows_indexes, cols_indexes)):
        fig.add_traces([
            go.Scatter(
                x=train_history.index + 1,
                y=train_history[metric],
                name="Training",
                line=dict(color="#636EFA"),
                showlegend=(i == 0),
            ),
            go.Scatter(
                x=validation_history.index + 1,
                y=validation_history[metric],
                name="Validation",
                line=dict(color="#EF553B"),
                showlegend=(i == 0),
            ),
        ], rows=row_indexes, cols=col_indexes)

        fig.update_xaxes(
            title="Epoch", range=[1, 20],
            row=row_indexes[0], col=col_indexes[0]
        )

        fig.update_yaxes(
            title=f"{metric.capitalize()}", range=[0, 1],
            row=row_indexes[0], col=col_indexes[0]
        )

    fig.update_layout(height=750, width=750)

    return fig

In [9]:
def testing_metrics_plot(test_results):

    metrics = ["loss", "f1-score", "fβ-score"]

    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=["AUC"], y=test_results["AUC_mean"],
            width=0.6, error_y=dict(array=[1.96 * test_results["AUC_std"]]),
        )
    )

    fig.add_trace(
        go.Bar(
            x=metrics, y=test_results[metrics].loc[0],
            width=0.6,
        )
    )

    fig.add_hline(
        y=0.9,
        line_width=0.75,
        line_dash="dash",
        line_color="green"
    )

    fig.update_layout(
        height=500, width=500,
        yaxis_range=[0, 1],
        showlegend=False
    )

    return fig

## **Plotting**

In [ ]:
def load_data(architecture, lead, format, purpose="train"):
    assert purpose in ["train", "validation", "test"], "Invalid purpose selected."

    directory = "testing" if purpose == "test" else "training"
    history = pd.read_csv(f"/content/drive/MyDrive/Thesis/{directory}/{format}/{architecture}_{format}_{purpose}.csv")

    return history

In [ ]:
ARCHITECTURES = ["lstm", "gru", "cnn_gru"]
LEADS = ["8leads", "12leads"]
FORMATS = ["raw", "filtered"]

experiments = chain(
    product(ARCHITECTURES, LEADS[1:], FORMATS[:1]),
    product(ARCHITECTURES, LEADS, FORMATS[1:])
)

for (architecture, lead, format) in tqdm(experiments):

    train_history = load_data(architecture, lead, format, purpose="train")
    validation_history = load_data(architecture, lead, format, purpose="validation")
    test_results = load_data(architecture, lead, format, purpose="test")

    experiment = "_".join([architecture, lead, format])

    fig = training_validation_metrics_plot(train_history, validation_history)
    fig.write_image(f"/content/drive/MyDrive/Thesis/visualizations/training_validation_plots/{experiment}.png")

    fig = testing_metrics_plot(test_results)
    fig.write_image(f"/content/drive/MyDrive/Thesis/visualizations/testing_plots/{experiment}.png")

In [ ]:
ARCHITECTURES = ["resnet18", "resnet34", "resnet50"]
FORMATS = ["spectrograms", "scaleograms"]

experiments = product(ARCHITECTURES, FORMATS)
experiments = list(experiments)

for (architecture, format) in tqdm(experiments):

    train_history = load_data(architecture, None, format, purpose="train")
    validation_history = load_data(architecture, None, format, purpose="validation")
    test_results = load_data(architecture, None, format, purpose="test")

    experiment = "_".join([architecture, format])

    fig = training_validation_metrics_plot(train_history, validation_history)
    fig.write_image(f"/content/drive/MyDrive/Thesis/visualizations/training_validation_plots/{experiment}.png")

    fig = testing_metrics_plot(test_results)
    fig.write_image(f"/content/drive/MyDrive/Thesis/visualizations/testing_plots/{experiment}.png")

100%|██████████| 6/6 [00:15<00:00,  2.62s/it]
